In [1]:
!nvidia-smi

Tue Sep 24 05:36:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# 1. Installing all the required Packages

In [3]:
! pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [4]:
# 2. Logged in with a Hugging Face account

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [6]:
# 3. Import all the required Libraries

In [7]:
!pip install langchain_community
from langchain.llms import HuggingFacePipeline


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00


In [8]:
# Autotokenizer : A tokenizer is responsible for preprocessing text into an array of numbers as inputs to a model.

In [9]:
from transformers import AutoTokenizer

In [10]:
import transformers

In [11]:
import torch

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
# 4. Load the Llama 2 model

In [14]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [15]:
tokenizer=AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [16]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [17]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs={'temperature':0})

In [18]:
prompt = "What would be a good name for a company that makes colorful socks"

In [19]:
print(llm(prompt))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


What would be a good name for a company that makes colorful socks?

Here are a few ideas to get you started:

1. SoleMates: This name plays on the idea of socks being a companion for your feet, and the word "mates" implies a sense of friendship or partnership.
2. Hue & Cry: This name combines the words "hue" (meaning color) and "cry" (meaning a loud or urgent call), which could work well for a company that specializes in bright and bold socks.
3. Toe-Tally Awesome: This name is a play on the phrase "totally awesome," which could work well for a company that wants to convey a sense of fun and excitement.
4. Sock It to Me: This name is a play on the phrase "sock it to me," which could work well for a company that wants to convey a sense of energy and excitement.
5. Footloose & Fancy-Free: This name combines the phrase "footloose" (meaning free and unencumbered) with "fancy-free" (meaning without any complicated or difficult situations), which could work well for a company that wants to c

In [20]:
prompt="I want to open a restaurant for  indian food. Suggest me a fancy name for this"

In [21]:
print(llm(prompt))

I want to open a restaurant for  indian food. Suggest me a fence name for this restaurant.

Answer:

Here are some suggestions for a restaurant name that serves Indian food:

1. Tandoor: Tandoor is a traditional Indian clay oven used for cooking naan bread, kebabs, and other dishes. Using this name for your restaurant will give it a distinctly Indian flavor.
2. Spice Route: This name evokes the idea of a journey through the diverse spices and flavors of India. It's catchy and memorable, and will likely appeal to customers looking for a taste of the exotic.
3. Mumbai Street Food: If you want to highlight the street food culture of India, this name is a great choice. Mumbai is known for its vibrant street food scene, and using this name will give your restaurant a distinctly urban and authentic feel.
4. Biryani House: Biryani is a popular Indian dish made with aromatic rice, meat, and spices. Using this name for your restaurant will give it a specific focus and will likely appeal to cust

In [22]:
# 05: Prompt Templates

Currently in the above applications we are writing an entire prompt, if you are creating a user directed application then this is not an ideal case

LangChain faciliates prompt management and optimization.

Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.


In many Large Language Model applications we donot pass the user input directly to the Large Language Model, we add the user input to a large piece of text called prompt template

In [23]:
# Importing all the required Libraries.

In [24]:
from langchain.prompts import PromptTemplate

In [25]:
# With the help of LLMChain we will combine PromptTemplate and LLM
from langchain.chains import LLMChain

In [26]:
# Example 1

In [43]:
prompt_template1 = PromptTemplate(input_variables=["cuisine"],
                                  template="I want to open a restaurant for {cuisine} food. Suggest a fency name for this")

In [44]:
input_prompt = prompt_template1.format(cuisine="Indian")

In [45]:
print(input_prompt)

I want to open a restaurant for Indian food. Suggest a fency name for this


In [46]:
chain = LLMChain(llm = llm , prompt=prompt_template1,verbose=True) # [ verbose=True ] means that the process will display detailed logs or outputs during execution,
                                                                   # showing what's happening step by step. It helps in debugging or understanding the internal flow of
                                                                   # how the LLMChain works, including how it processes the input and generates the response. You'll see the
                                                                   # intermediate steps and outputs printed to the console or logs.
response = chain.run("Indian")
print(response)

Prompt after formatting:
I want to open a restaurant for Indian food. Suggest a fency name for this

> Finished chain.
I want to open a restaurant for Indian food. Suggest a fency name for this restaurant.
by Anonymous 1 year ago
I want to open a restaurant for Indian food. Suggest a fancy name for this restaurant.
I would suggest the following fancy names for your Indian restaurant:
1. Tandoor Nights - This name suggests a cozy and intimate setting where you can enjoy delicious Indian food cooked in a tandoor oven.
2. Spice Route - This name takes you on a journey through the flavors of India, with a nod to the spice routes that have connected the subcontinent to the world for centuries.
3. Mumbai Street Food Co. - This name captures the vibrant and bustling atmosphere of India's street food culture, with a focus on the flavors and spices of Mumbai.
4. Rajasthani Royal - This name suggests a restaurant that serves traditional Rajasthani cuisine in a regal and luxurious setting.
5. Tik

In [33]:
# Example 2

In [36]:
prompt_template2 = PromptTemplate(input_variables=["book_name"],
                                  template="Provide me a concise summary of the book {book_name}")

In [37]:
input_prompt = prompt_template2.format(book_name = "Rich Dad Poor Dad")

In [38]:
print(input_prompt)

Provide me a concise summary of the book Rich Dad Poor Dad


In [39]:
chain = LLMChain(llm = llm , prompt=prompt_template2,verbose=True)
response = chain.run("Rich Dad Poor Dad")
print(response)

Prompt after formatting:
Provide me a concise summary of the book Rich Dad Poor Dad

> Finished chain.
Provide me a concise summary of the book Rich Dad Poor Dad by Robert Kiyosaki

Robert Kiyosaki's book "Rich Dad Poor Dad" is a personal finance guide that challenges conventional beliefs about money and wealth. Here's a concise summary of the book:

1. The Rich Don't Work for Money: The book's central message is that money is a tool, not the ultimate goal. The rich focus on building assets that generate passive income, rather than working for a paycheck.
2. The Poor and Middle Class Work for Money: The poor and middle class work for money, but they don't own assets that generate passive income. They are trapped in the cycle of working for a living, with little hope of achieving financial freedom.
3. Asset vs. Liability: The rich invest in assets that appreciate in value, such as real estate, stocks, and businesses. The poor and middle class, on the other hand, invest in liabilities, s